In [1]:
from time import sleep
import cv2
from sklearn.cluster import KMeans
import numpy as np
from skimage import io
import spotipy
import imutils
import os
from IPython.display import clear_output
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
#use k-means in lab colorspace to determine dominant colors
def dominantColorsLAB(img,clusters):
    
    #convert to lab from bgr
    img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    
            
    #reshaping to a list of pixels
    img = img.reshape((img.shape[0] * img.shape[1], 3))
    
    
    #using k-means to cluster pixels, set seed so that colors are deterministic
    kmeans = KMeans(n_clusters=clusters,max_iter=300,tol=1e-3,random_state=42)
    kmeans.fit(img)
    
    colors = kmeans.cluster_centers_

    return colors.astype(int)

In [3]:
#get colors for light [0],[1] would be primary colors, [2] a trim color, and [3] a background color
def getLights(img):
    #check for desired colors on various incrementing cluster sizes
    for clusters in range(4,10):

        #run the k-means in LAB color space
        colors = dominantColorsLAB(img, clusters).tolist()

        #try to find 3 bright colors, [0] is the L in LAB
        brightColors = list(filter(lambda x: x[0] > 110, colors))
        if(len(brightColors) < 3):
            continue

        #only take the top 3 bright colors as returned from the k-means
        brightColors = brightColors[:3]

        #set bg color to be the most dominant color cluster remaining
        bgColor =[0,0,0]
        for color in colors:
            if(color not in brightColors):
                bgColor = color
                break
        brightColors.append(bgColor)

        print(clusters)

        #convert to rgb before returning
        return cv2.cvtColor(np.uint8([brightColors]), cv2.COLOR_LAB2RGB)[0]
    
    #fallthrough
    print("Failed to find desired colors")

    #just get the 4 most dominant colors and sort them by brightness decending
    failColors = dominantColorsLAB(img, 4) 
    failColors = sorted(failColors,key=lambda x:x[0],reverse=True)
    
    #convert to rgb before returning
    return cv2.cvtColor(np.uint8([failColors]), cv2.COLOR_LAB2RGB)[0]

        
            
        




In [4]:
#get spotify info from text file
secretFile = open('secret.txt')

os.environ['SPOTIPY_CLIENT_ID'] = secretFile.readline().strip()
os.environ['SPOTIPY_CLIENT_SECRET'] = secretFile.readline().strip()
os.environ['SPOTIPY_REDIRECT_URI'] = secretFile.readline().strip()

userID = secretFile.readline().strip()
secretFile.close()

#spotipy OAuth should automatically handle token refresh
spotify = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(scope='user-read-currently-playing',cache_path='.cache-'+userID))

lastURL = ""

In [ ]:
while(True):

    #token should refresh automatically
    track = spotify.current_user_playing_track()

    #handle bad track objects and rate limiting
    try:
        imageURL = track['item']['album']['images'][0]['url']
    except TypeError:
        imageURL = lastURL
        sleep(1)

    #if the song has changed
    if(lastURL != imageURL):
        lastURL = imageURL

        img = imutils.url_to_image(lastURL)

        #optimization for speed
        img  = cv2.resize(img,(300,300))
        
        #make convert opencv BGR image to RGB for notebook
        imgRBG = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        #display album in notebook
        plt.imshow(imgRBG)
        plt.show()
        
        #get color palette
        colorsLAB = getLights(img)
        paletteLAB = np.array(colorsLAB, dtype=np.uint8)

        
        #display a color swatch
        display = np.array([[0,1],[2,3]], dtype=np.uint8)
        io.imshow(paletteLAB[display])
        io.show()

        #clear the notebook
        clear_output(wait=True)

    #spotify api is fine with 3600 req/hour
    sleep(1)